Experiment: BERT on full_dataset
------
For this experiment we assume that the dataset is stored in a pkl file.

In [1]:
import sys; sys.path.append('../..')
import torch
from bias_classification import prepare_model 
from src.utils import *

%load_ext autoreload
%autoreload 2

In [2]:
params = prepare_model.intialize_params("experiment_params.json")

In [3]:
import pickle
dataset = pickle.load(open('dataset.pkl', 'rb'))

In [4]:
# Run to print out the current key names in the dataset
dataset.get_key_names()

dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'pos', 'pos_weak_labels', 'bias_label'])

In [5]:
tok2id = prepare_model.get_tok2id(params.intermediary_task)

06/07/2019 18:25:50 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [6]:
print(tok2id['[CLS]'])

101


In [7]:
original_tokids = dataset.get_val("pre_ids")

In [8]:
rows, cols = original_tokids.shape

In [9]:
new_col = tok2id['[CLS]'] * torch.ones(rows,1, dtype=torch.long)

In [10]:
print(new_col.shape)
print(original_tokids.shape)

torch.Size([51444, 1])
torch.Size([51444, 80])


In [11]:
tok_ids = torch.cat((new_col,original_tokids), dim=1)[:,:80]

In [12]:
dataset.add_data(tok_ids, 'pre_ids')

In [13]:
mask = dataset.get_val("masks")

In [14]:
mask = -1 * (mask.to(torch.int8) - 1)

In [15]:
for row in range(len(dataset)):
    idx = mask[row].tolist().index(0)
    if idx == 80:
        continue
    mask[row,idx] = 1

In [16]:
mask = mask.to(torch.uint8)

In [17]:
dataset.add_data(mask, 'masks')

Run the cell below to reset params 
-----

In [18]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

06/07/2019 17:37:08 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /sailhome/rdm/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
06/07/2019 17:37:08 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /sailhome/rdm/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpzpcfgca7
06/07/2019 17:37:12 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 

In [19]:
print(params.final_task)

{'model': 'bert_basic_uncased_sequence', 'output_dim': 1, 'data_split': {'train_split': 0.001, 'eval_split': 0.001, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 2, 'batch_size': 64, 'lr': 0.001}}


Boostrapping procedure 

In [20]:
bert_args = {'attention_mask_key': 'masks', 'num_bootstrap_iters':20}
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='pre_ids', label_key='bias_label', threshold=0.42, **bert_args)

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
tensor([[  101, 21146, 18569,  ...,     0,     0,     0],
        [  101,  1996,  4035,  ...,     0,     0,     0],
        [  101, 10424, 20518,  ...,     0,     0,     0],
        ...,
        [  101,  2077,  3352,  ...,  1012,     0,     0],
        [  101,  2005,  2012,  ...,     0,     0,     0],
        [  101,  2648,  3834,  ...,     0,     0,     0]], device='cuda:0')
0.7010625600814819


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
tensor([[  101, 21146, 18569,  ...,     0,     0,     0],
        [  101,  1996,  4035,  ...,     0,     0,     0],
        [  101, 10424, 20518,  ...,     0,     0,     0],
        ...,
        [  101,  2077,  3352,  ...,  1012,     0,     0],
        [  101,  2005,  2012,  ...,     0,     0,     0],
        [  101,  2648,  3834,  ...,     0,     0,     0]], device='cuda:0')
1.6438924074172974


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
tensor([[ 101, 1996, 3278,  ...,    0,    0,    0],
        [ 101, 2002, 2001,  ...,    0,    0,    0],
        [ 101, 1996, 4598,  ...,    0,    0,    0],
        ...,
        [ 101, 2002, 2052,  ...,    0,    0,    0],
        [ 101, 2012, 1996,  ..., 5143, 5196, 1012],
        [ 101, 2065, 1996,  ...,    0,    0,    0]], device='cuda:0')
0.7005409598350525


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
tensor([[ 101, 1996, 3278,  ...,    0,    0,    0],
        [ 101, 2002, 2001,  ...,    0,    0,    0],
        [ 101, 1996, 4598,  ...,    0,    0,    0],
        ...,
        [ 101, 2002, 2052,  ...,    0,    0,    0],
        [ 101, 2012, 1996,  ..., 5143, 5196, 1012],
        [ 101, 2065, 1996,  ...,    0,    0,    0]], device='cuda:0')
0.7091359496116638


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
tensor([[  101,  1996,  2958,  ...,     0,     0,     0],
        [  101,  1037, 25908,  ...,     0,     0,     0],
        [  101,  2174,  1010,  ...,     0,     0,     0],
        ...,
        [  101,  2048,  5611,  ...,     0,     0,     0],
        [  101,  1999,  2337,  ...,     0,     0,     0],
        [  101,  2012,  5096,  ...,     0,     0,     0]], device='cuda:0')
0.6845828890800476


...

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
tensor([[  101,  1996,  3287,  ...,     0,     0,     0],
        [  101,  1996,  9935,  ...,     0,     0,     0],
        [  101,  2016, 18807,  ...,     0,     0,     0],
        ...,
        [  101,  1996,  4668,  ...,     0,     0,     0],
        [  101,  4487,  7361,  ...,     0,     0,     0],
        [  101,  2130,  3282,  ...,     0,     0,     0]], device='cuda:0')
0.6644759178161621


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
tensor([[  101,  1996,  3287,  ...,     0,     0,     0],
        [  101,  1996,  9935,  ...,     0,     0,     0],
        [  101,  2016, 18807,  ...,     0,     0,     0],
        ...,
        [  101,  1996,  4668,  ...,     0,     0,     0],
        [  101,  4487,  7361,  ...,     0,     0,     0],
        [  101,  2130,  3282,  ...,     0,     0,     0]], device='cuda:0')
0.9123970866203308


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
tensor([[  101, 12312,  2078,  ...,     0,     0,     0],
        [  101, 15764,  2609,  ...,     0,     0,     0],
        [  101, 28293,  1005,  ...,     0,     0,     0],
        ...,
        [  101,  2530,  5784,  ...,     0,     0,     0],
        [  101,  2023, 23273,  ...,     0,     0,     0],
        [  101,  2116,  2111,  ...,     0,     0,     0]], device='cuda:0')
0.6871629357337952


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
tensor([[  101, 12312,  2078,  ...,     0,     0,     0],
        [  101, 15764,  2609,  ...,     0,     0,     0],
        [  101, 28293,  1005,  ...,     0,     0,     0],
        ...,
        [  101,  2530,  5784,  ...,     0,     0,     0],
        [  101,  2023, 23273,  ...,     0,     0,     0],
        [  101,  2116,  2111,  ...,     0,     0,     0]], device='cuda:0')
0.8954633474349976


KeyboardInterrupt: 

In [ ]:
statistics

New run 

## No bootstrapping -- cannot calculate 95% confidence intervals 

In [18]:
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

06/07/2019 18:25:58 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /sailhome/rdm/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
06/07/2019 18:25:58 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /sailhome/rdm/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmphmd8w9ie
06/07/2019 18:26:02 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 

In [19]:
data_split = params.final_task['data_split']
batch_size = params.final_task['training_params']['batch_size']
train_dataloader, eval_dataloader, test_dataloader = dataset.split_train_eval_test(**data_split, batch_size=batch_size)

In [20]:
bert_args = {'attention_mask_key': 'masks', 'num_bootstrap_iters':20}
losses, evaluations = classification_experiment.train_model(train_dataloader, eval_dataloader, input_key='pre_ids', label_key='bias_label', threshold=0.42, **bert_args)

tensor([[0.1931],
        [0.3275],
        [0.4975],
        [0.5178],
        [0.3146]], device='cuda:0', grad_fn=<ThAddmmBackward>)
0.5898348689079285


tensor([[1.7519],
        [1.2113],
        [1.8361],
        [1.3548],
        [1.0269]], device='cuda:0', grad_fn=<ThAddmmBackward>)
0.42622146010398865


tensor([[ 1.9493],
        [ 2.4012],
        [ 2.4119],
        [ 2.5295],
        [-0.9420]], device='cuda:0', grad_fn=<ThAddmmBackward>)
0.14231112599372864


tensor([[-0.3927],
        [ 0.5768],
        [-1.2920],
        [-0.2987],
        [-2.6147]], device='cuda:0', grad_fn=<ThAddmmBackward>)
0.7626725435256958


tensor([[5.4913],
        [5.7815],
        [5.4742],
        [5.4188],
        [5.3741]], device='cuda:0', grad_fn=<ThAddmmBackward>)
1.0788997411727905


...

tensor([[1.3387],
        [1.5914],
        [1.7327],
        [1.0715],
        [1.3729]], device='cuda:0', grad_fn=<ThAddmmBackward>)
0.49485453963279724


tensor([[1.0766],
        [1.5010],
        [1.3766],
        [1.2010],
        [1.1305]], device='cuda:0', grad_fn=<ThAddmmBackward>)
0.4785802960395813


tensor([[1.3106],
        [1.1552],
        [1.1379],
        [1.2731],
        [1.0521]], device='cuda:0', grad_fn=<ThAddmmBackward>)
0.47781306505203247


tensor([[1.1489],
        [1.4846],
        [1.5582],
        [1.4071],
        [1.4229]], device='cuda:0', grad_fn=<ThAddmmBackward>)
0.5056911110877991


KeyboardInterrupt: 

In [ ]:
losses

In [ ]:
avg_losses = [average_data(epoch_losses) for epoch_losses in losses]
avg_predictions = [average_data(epoch_evaluations) for epoch_evaluations in evaluations]

In [ ]:
min_loss, max_loss, avg_loss = get_statistics(avg_losses, "loss")
print(min_loss)

In [ ]:
min_auc, max_auc, avg_auc = get_statistics(avg_predictions, "auc")

In [ ]:
print(max_auc)

In [ ]:
print(min_auc)